# Goal

To combine [tinystories](https://arxiv.org/abs/2305.07759), [text-to-sql](https://huggingface.co/datasets/b-mc2/sql-create-context), and [textbooks are all you need datasets](https://ar5iv.labs.arxiv.org/html/2306.11644), into one dataset to train an encoder-decoder Transformer model, for text-to-code tasks. All three are on Huggingface, to avoid data ingestion pains for now. 

To then tokenise this dataset via SentencePiece.

# Imports

In [ ]:
import sys
import einops
from dataclasses import dataclass
from transformer_lens import HookedTransformer
from transformer_lens.utils import gelu_new, tokenize_and_concatenate
import torch as t
from torch import Tensor
import torch.nn as nn
import numpy as np
import math
from tqdm.notebook import tqdm
from typing import Tuple, List, Optional, Dict
from jaxtyping import Float, Int
from transformers.models.gpt2.tokenization_gpt2_fast import GPT2TokenizerFast
from collections import defaultdict
from rich.table import Table
from rich import print as rprint
import datasets
from torch.utils.data import DataLoader
import wandb
from pathlib import Path
import webbrowser
from datasets import load_dataset
import sentencepiece as spm
from datasets import concatenate_datasets

# Load three datasets

Note: login to huggingface-cli on command line to download textbooks dataset

In [ ]:
# Run below on command line if it doesn't work here
# Generate token from HF
# !huggingface-cli login

In [ ]:
# Pull tinystories from HF
tiny_stories = load_dataset('roneneldan/TinyStories')

text_to_sql = load_dataset('b-mc2/sql-create-context')

textbooks_all_you_need = load_dataset('nampdn-ai/tiny-codes')

In [ ]:
textbooks_all_you_need.keys()

# Combine/Prepare for SentencePiece

In [ ]:
tiny_stories_train = tiny_stories['train']
text_to_sql_train = text_to_sql['train']
textbooks_all_you_need_train = textbooks_all_you_need['train']

print (tiny_stories_train.features)
print (text_to_sql_train.features)
print (textbooks_all_you_need_train.features)
print (len(tiny_stories_train))
print (len(text_to_sql_train))
print (len(textbooks_all_you_need_train))

Create mini versions of each dataset for testing:

In [ ]:
tiny_stories_train_testing = tiny_stories_train.shuffle().select(range(1000))
text_to_sql_train_testing = text_to_sql_train.shuffle().select(range(1000))
textbooks_all_you_need_train_testing = textbooks_all_you_need_train.shuffle().select(range(1000))


I'm going to have to feed one 'language' of data into the encoder, and the other into the decoder. Languages = (English, code)? Or is it (English, SQL, Python, Java...)?

This will be way too complicated a task, let's just use SQL. 

In [ ]:
# Define a filter function
def filter_sql_entries(example):
    return 'sql' in example['programming_language'].lower()

# Apply the filter function
textbooks_all_you_need_train_sql = textbooks_all_you_need_train.filter(filter_sql_entries)
textbooks_all_you_need_train_testing_sql = textbooks_all_you_need_train_testing.shuffle().filter(filter_sql_entries)


In [ ]:
print (len(textbooks_all_you_need_train_sql))
print (len(textbooks_all_you_need_train_testing_sql))

# Store as individual sentences

In [ ]:
# # Testing of SP Input
# # Combine the relevant fields from each dataset into a single text file for SentencePiece training
# with open('SP_data_encoder_decoder_testing.txt', 'w', encoding='utf-8') as f:
#     for example in tiny_stories_train_testing:
#         f.write(example['text'].replace('\n', '') + '\n')

#     print ('tiny_stories_train done')
#     for example in text_to_sql_train_testing:
#         f.write(example['context'] + '\n')
#         f.write(example['question'] + '\n')
#         f.write(example['answer'] + '\n')  # This is typically the target language in translation tasks
#     print ('text_to_sql_train done')
#     for example in textbooks_all_you_need_train_testing:
#         f.write('\n'.join([example[field] for field in example if field not in ['idx', 'response']]) + '\n')

# #         f.write(' '.join([example[field] for field in example if field not in ['idx', 'response']]) + '\n')
#         f.write(example['response'] + '\n')  # Include the 'response' field as it is part of the target language

In [ ]:
# Combine the relevant fields from each dataset into a single text file for SentencePiece training
with open('SP_data_encoder_decoder.txt', 'w', encoding='utf-8') as f:
    for example in tiny_stories_train:
        f.write(example['text'].replace('\n', '') + '\n')
    print ('tiny_stories_train done')
    for example in text_to_sql_train:
        f.write(example['context'] + '\n')
        f.write(example['question'] + '\n')
        f.write(example['answer'] + '\n')  # This is typically the target language in translation tasks
    print ('text_to_sql_train done')
    for example in textbooks_all_you_need_train:
        f.write(example['prompt'] + '\n')
        f.write(example['response'] + '\n')  # Include the 'response' field as it is part of the target language

# Train SentencePiece

Initial runs crashing notebook. Restrict rows used. First get number of rows in data 

In [ ]:
def count_rows(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        line_count = sum(1 for line in file)
    return line_count

# Example usage
num_rows = count_rows('SP_data_encoder_decoder.txt')  # Replace 'your_file.txt' with your file path
print(f"Number of rows in the file: {num_rows}")

In [ ]:
import sentencepiece as spm

# Define parameters for training
train_args = {
    'input': 'SP_data_encoder_decoder.txt',             # Input file
    'model_prefix': 'SP_encoder_decoder_model',        # Prefix for the output model files (.model and .vocab)
    'vocab_size': 32000,              # Size of the vocabulary
    'character_coverage': 0.9997,     # Character coverage to be considered for the model. Good defaults are: 0.9995 for languages with rich character sets like Japanese or Chinese and 0.9997 for others
    'model_type': 'unigram',        # Model type can be 'unigram' (default), 'bpe', 'char', or 'word'
    'input_sentence_size': 1300000,
    'shuffle_input_sentence': True,
    'pad_id': 0,
    'unk_id': 1,
    'bos_id': 2,
    'eos_id': 3,
    'pad_piece': '[PAD]',
    'unk_piece': '[UNK]',
    'bos_piece': '[BOS]',
    'eos_piece': '[EOS]'}

# Train the model
spm.SentencePieceTrainer.Train(' '.join([f'--{k}={v}' for k, v in train_args.items()]))

print("Model trained and saved as mymodel.model and mymodel.vocab!")

# Check SP trained well

The sentencepiece call above will now:
1. identify [BOS] tokens as beginning a sentence, same for [EOS] and end of sentence
2. Replace unknown tokens with [UNK]/token ids with 1 
3. Replace padding tokens with [PAD], and token ids with 0

To check:
What is the token encoding for a sample of rows in each dataset? 

In [ ]:
# Initialize SentencePiece processor and load your model
sp = spm.SentencePieceProcessor()
sp.load('SP_encoder_decoder_model.model')  # Replace with your model file

# Your input string
input_string =     '''
    cin >> consentGiven;

    // Based on the user's answer, display appropriate instructions
    if (consentGiven) {
        cout << endl
'''

input_string_2 =     '''
cin >> consentGiven; // Based on the user's answer, display appropriate instructions if (consentGiven) {cout << endl
'''

# Encode the string into SentencePiece tokens
encoded_pieces = sp.EncodeAsPieces(input_string)
print("Encoded as pieces:", encoded_pieces)

# Alternatively, encode the string into token IDs
encoded_ids = sp.EncodeAsIds(input_string)
print("Encoded as ids:", encoded_ids)

# Encode the string into SentencePiece tokens
encoded_pieces_2 = sp.EncodeAsPieces(input_string_2)
print("Encoded as pieces:", encoded_pieces_2)

# Alternatively, encode the string into token IDs
encoded_ids_2 = sp.EncodeAsIds(input_string_2)
print("Encoded as ids:", encoded_ids_2)

Ok, looks alright over the SQL! Will assume it's fine over tinystories too then, not worried about that.

# Create training datasets for encoder and decoder

For this, we'll have:
1. separate out the inputs (natural language plus SQL contextual?) and the outputs (resultant SQL?). 
2. We'll also have to append BOS, EOS and Padding tokens, just as our trained SP model expects. 
3. We'll have to create datasets and dataloaders objects for the inputs and outputs separately too. 
4. We'll also need collate functions to pad each batch. 

Input dataset:
1. Encoder: tiny_stories['text'], Decoder: X 
2. text_to_sql['question'] + text_to_sql['context'], Decoder: text_to_sql['answer']
3. textbooks_all_you_need['prompt']: textbooks['response']

Maybe start with just (2) and (3) for now. In fact let's ignore context from (2) also.

- Append BOS and EOS tokens to target sequence/decoder input
- Do the same for input sequences/encoder input? Definitely for EOS token, probably doesn't matter for BOS for encoder (but the q/k/v matrices will be different size though. So only use EOS token?)
- Full target sequence is [BOS, 1,2,3,4, EOS] 
- decoder input is [BOS, 1,2,3,4]
- Decoder will output predictions on [1,2,3,4,EOS], so use [1,2,3,4,EOS] in loss calculation

In [ ]:
class EncoderDecoderDataset(Dataset):
    def __init__(self, hf_dataset):
        self.hf_dataset = hf_dataset

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        # This method should return a single sample at a time
        item = self.hf_dataset[idx]
        # Process the item (e.g., tokenization, numericalization) as required
        # ...
        return item

In [ ]:
from datasets import Features, Value

def standardize_types(example):
    example['prompt'] = str(example['prompt'])
    example['response'] = str(example['response'])
    return example



text_to_sql_train_mapped = text_to_sql_train.map(lambda example: {'prompt': example['question'], 
                                         'response': example['answer']}, 
                        remove_columns=['question', 'context', 'answer'])

cols_to_keep = {'prompt', 'response'}

columns_to_remove = [col for col in textbooks_all_you_need_train_sql.column_names if col not in cols_to_keep]

textbooks_all_you_need_train_sql_trimmed = textbooks_all_you_need_train_sql.map(lambda example: example, remove_columns=columns_to_remove)

textbooks_all_you_need_train_sql_trimmed_2 = textbooks_all_you_need_train_sql_trimmed.cast(
    Features({"response": Value("string"), "prompt": Value("string")}))
combined_dataset = concatenate_datasets([text_to_sql_train_mapped, textbooks_all_you_need_train_sql_trimmed_2])

In [ ]:
print (len(textbooks_all_you_need_train_sql_trimmed))
print (len(text_to_sql_train_mapped))
print (len(combined_dataset))

for i in range(5):
    print (combined_dataset[i])
    print (combined_dataset[-i])

Looks like dataset concatenation was successful!

# Create Dataset Class for encoder-decoder

In [ ]:
class EncoderDecoderDataset(t.utils.data.Dataset):
    def __init__(self, input_data, sp_tokenizer_model):
        self.column_names = ['prompt', 'response']
        self.data = input_data
        self.tokenizer = sp_tokenizer_model
        
    def __len__(self):
        return (len(self.data))
    
    def __getitem__(self, idx):
        row = self.data[idx]
        model_input = str(self.tokenizer.sp.bos_id()) 
        + ' ' + self.tokenizer.encode(row['[prompt]']) 
        + ' ' + str(self.tokenizer.sp.eos_id())
        
        ground_truth = str(self.tokenizer.sp.bos_id()) 
        + ' ' + self.tokenizer.encode(row['[response]']) 
        + ' ' + str(self.tokenizer.sp.eos_id())
        return {
            'model_input': model_input,
            'ground_truth': ground_truth,
            'tensor_model_input': torch.tensor(model_input),
            'tensor_ground_truth': torch.tensor(ground_truth)
        }
        
    def collate_fn(self, batch):
        # This should really be using [PAD], but easier to directly add 0s, as 0 is PAD ID from tokenizer
        input_pad = t.nn.utils.rnn.pad_sequence([item['model_input'] for item in batch], batch_first=True, padding_value=0)
        label_pad = t.nn.utils.rnn.pad_sequence([item['ground_truth'] for item in batch], batch_first=True, padding_value=0)
        
        return {
            'model_inputs': [item['model_input'] for item in batch],
            'ground_truths': [item['ground_truth'] for item in batch],
            'tensor_model_input': input_pad,
            'tensor_ground_truth': label_pad
        }

# Create dataloaders

In [ ]:
EncoderDecoderDataset = EncoderDecoderDataset(
    combined_dataset, 
    sp.load('SP_encoder_decoder_model.model'))

EncoderDecoderDataLoader = t.utils.data.DataLoader(
    EncoderDecoderDataset,
    batch_size=64,
    shuffle=True,
    collate_fn=EncoderDecoderDataset.collate_fn)


# Create Encoder-Decoder Model

Adjustments to make, to Transformer Code, to implement encoder-decoder:
1. Create separate encoder and decoder transformer classes, replacing the TransformerBlock and DemoTransformer classes. 
2. Implement cross-attention, after self-attention, in the decoder block

## Dataclass

In [ ]:
from dataclasses import dataclass
@dataclass
class Config:
    d_model: int = 768
#     debug: bool = True
    layer_norm_eps: float = 1e-5
    d_vocab: int = 50257
    init_range: float = 0.02
    n_ctx: int = 1024
    d_head: int = 64
    d_mlp: int = 3072
    n_heads: int = 12
    n_layers: int = 12

cfg = Config()
print(cfg)

## Device

In [ ]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")


## Embedding Modules

In [ ]:
import torch as t
class Embed(nn.Module):
    def __init__(self, cfg:Config):
        super().__init__()
        self.cfg = cfg
        self.W_E = nn.Parameter(t.empty(cfg.d_vocab, cfg.d_model))
        nn.init.normal_(self.W_E, std = self.cfg.init_range)
    def forward(self, tokens: Int[Tensor, "batch position"]) -> Float[Tensor, "batch position d_model"]:
        return self.W_E[tokens]
    
class PosEmbed(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_pos = nn.Parameter(t.empty(cfg.n_ctx, cfg.d_model))
        nn.init.normal_(self.W_pos, std=self.cfg.init_range)
        
    def forward(self, tokens: Int[Tensor, "batch position"]) -> Float[Tensor, "batch position d_model"]:
        batch, seq_len = tokens.shape
        return einops.repeat(self.W_pos[:seq_len], "seq d_model -> batch seq d_model", batch = batch)

## Transformer Modules

## Attention

In [ ]:
class Attention(nn.Module):
    IGNORE: Float[Tensor, ""]
    
    def __init__(self, cfg:Config, is_causal: bool, decoder = True):
        super().__init__()
        self.cfg = cfg 
        self.W_Q = nn.Parameter(t.empty(cfg.n_heads, cfg.d_model, cfg.d_head))
        self.W_K = nn.Parameter(t.empty(cfg.n_heads, cfg.d_model, cfg.d_head))
        self.W_V = nn.Parameter(t.empty(cfg.n_heads, cfg.d_model, cfg.d_head))
        self.W_O = nn.Parameter(t.empty(cfg.n_heads, cfg.d_head, cfg.d_model))
        self.b_Q = nn.Parameter(t.zeros((cfg.n_heads, cfg.d_head)))
        self.b_K = nn.Parameter(t.zeros((cfg.n_heads, cfg.d_head)))
        self.b_V = nn.Parameter(t.zeros((cfg.n_heads, cfg.d_head)))
        self.b_O = nn.Parameter(t.zeros((cfg.d_model)))
        nn.init.normal_(self.W_Q, std=self.cfg.init_range)
        nn.init.normal_(self.W_K, std=self.cfg.init_range)
        nn.init.normal_(self.W_V, std=self.cfg.init_range)
        nn.init.normal_(self.W_O, std=self.cfg.init_range)
        self.register_buffer("IGNORE", t.tensor(-1e5, dtype=t.float32, device=device))
        self.key_activations = t.empty(cfg.n_heads, cfg.d_model, cfg.d_head)
        self.value_activations = t.empty(cfg.n_heads, cfg.d_model, cfg.d_head)

    def forward(
        self, normalized_resid_pre: Float[Tensor, "batch seq_len d_model"]
    ) -> Float[Tensor, "batch seq_len d_model"]:
        Keys = einops.einsum(
            normalized_resid_pre,
            self.W_K,
            "batch seq_len d_model, n_heads d_model d_head -> batch seq_len n_heads d_head"
            ) + self.b_K
        
        if (decoder and is_causal):
            Keys = Keys
        elif (decoder and not is_causal):
            Keys = self.key_activations
        elif (not decoder and not is_causal):
            self.key_activations = Keys
        else:
            print ('Error in attention at key activation bools!')

        Queries = einops.einsum(
            normalized_resid_pre,
            self.W_Q,
            "batch seq_len d_model, n_heads d_model d_head -> batch seq_len n_heads d_head"
            ) + self.b_Q
        
        Values = einops.einsum(
            normalized_resid_pre,
            self.W_V,
            "batch seq_len d_model, n_heads d_model d_head -> batch seq_len n_heads d_head"
            ) + self.b_V
        
        if (decoder and is_causal):
            Values = Values
        elif (decoder and not is_causal):
            Values = self.key_activations
        elif (not decoder and not is_causal):
            self.value_activations = Values
        else:
            print ('Error in attention at value activation bools!')
                
        Attention_Scores = einops.einsum(
            Queries,
            Keys,
            "batch seq_len_Q n_heads d_head, batch seq_len_K n_heads d_head -> batch n_heads seq_len_Q seq_len_K")
        
        # Make Attention dependant on is_causal bool
        # for encoder-decoder, which uses both masked and unmasked attention
        if (is_causal):
            Attention_Scores_Scaled = self.apply_causal_mask(Attention_Scores / self.cfg.d_head**0.5)
        else:
            Attention_Scores_Scaled = Attention_Scores / self.cfg.d_head**0.5
            
        Attention_Scores_Scaled_Softmaxed = Attention_Scores_Scaled.softmax(-1)

        Z = einops.einsum(
            Values,
            Attention_Scores_Scaled_Softmaxed,
            "batch seq_len_K n_heads d_head, batch n_heads seq_len_Q seq_len_K -> batch seq_len_Q n_heads d_head")

        Attention_Out = einops.einsum(
            Z, 
            self.W_O, 
            "batch seq_len_Q n_heads d_head, n_heads d_head d_model -> batch seq_len_Q d_model"
            ) + self.b_O
        
        return Attention_Out
    
    def apply_causal_mask(
        self, attn_scores: Float[Tensor, "batch n_heads query_pos key_pos"]
    ) -> Float[Tensor, "batch n_heads query_pos key_pos"]:
        '''
        Applies a causal mask to attention scores, and returns masked scores.
        '''
        key_by_query_ones = t.ones(attn_scores.size(-2), attn_scores.size(-1), device = attn_scores.device)
        mask = t.triu(key_by_query_ones, diagonal = 1).bool()
        attn_scores.masked_fill(mask, self.IGNORE)
        return attn_scores

## MLP

In [ ]:
class MLP(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_in = nn.Parameter(t.empty(cfg.d_model, cfg.d_mlp))
        self.W_out = nn.Parameter(t.empty(cfg.d_mlp, cfg.d_model))
        self.b_in = nn.Parameter(t.zeros(cfg.d_mlp))
        self.b_out = nn.Parameter(t.zeros(cfg.d_model))
        nn.init.normal_(self.W_in, std = self.cfg.init_range)
        nn.init.normal_(self.W_out, std = self.cfg.init_range)
    
    def forward(
        self, normalized_resid_mid: Float[Tensor, "batch seq_len d_model"]
    ) -> Float[Tensor, "batch seq_len d_model"]:
        
        post_W_in = einops.einsum(
            normalized_resid_mid,
            self.W_in,
            "batch seq_len d_model, d_model d_mlp -> batch seq_len d_mlp") + self.b_in
        
        post_activation = gelu_new(post_W_in) 
        
        post_W_out = einops.einsum(
            post_activation,
            self.W_out, 
            "batch seq_len d_mlp, d_mlp d_model -> batch seq_len d_model") + self.b_out
        return post_W_out

## LayerNorm

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.w = nn.Parameter(t.ones(cfg.d_model))
        self.b = nn.Parameter(t.zeros(cfg.d_model))

    def forward(self, residual: Float[Tensor, "batch posn d_model"]) -> Float[Tensor, "batch posn d_model"]:
        residual_mean = residual.mean(dim=-1, keepdim=True)
        residual_std = (residual.var(dim=-1, keepdim=True, unbiased=False) + self.cfg.layer_norm_eps).sqrt()

        residual = (residual - residual_mean) / residual_std
        return residual * self.w + self.b

## Assemble Transformer Block

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.ln1 = LayerNorm(cfg)
        self.attn = Attention(cfg, is_causal=True)
        self.ln2 = LayerNorm(cfg)
        self.mlp = MLP(cfg)
    
    def forward(
        self, resid_pre: Float[Tensor, "batch seq_len d_model"]
    ) -> Float[Tensor, "batch seq_len d_model"]:
        attention
        resid_mid = self.attn(self.ln1(resid_pre)) + resid_pre
        resid_post = self.mlp(self.ln2(resid_mid)) + resid_mid
        return resid_post

## Assemble Encoder Block

In the encoder block, we just need to remove the causal mask, and then output the key and value activations, on a certain input. 

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.ln1 = LayerNorm(cfg)
        self.attn = Attention(cfg, is_causal=False, decoder = False)
        self.ln2 = LayerNorm(cfg)
        self.mlp = MLP(cfg)

    def forward(
        self, resid_pre: Float[Tensor, "batch seq_len d_model"]
    ) -> Float[Tensor, "batch seq_len d_model"]:
            
        attention_activations = self.attn(self.ln1(resid_pre))
        resid_mid = attention_activations + resid_pre

        mlp_activations = self.mlp(self.ln2(resid_mid))
        resid_post = mlp_activations + resid_mid
        return resid_post, attention_activations.key_activations, attention_activations.value_activations

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self,
                 cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.ln1 = LayerNorm(cfg)
        self.attn = Attention(cfg, is_causal=True, decoder = True)
        self.ln2 = LayerNorm(cfg)
        self.attn2 = Attention(cfg, is_causal=False, decoder = True)
        self.ln3 = LayerNorm(cfg)
        self.mlp = MLP(cfg)
    
    def forward(
        self,
        resid_pre: Float[Tensor, "batch seq_len d_model"],
        key_activations: Float[Tensor, "batch seq_len n_heads d_head"],
        value_activations: Float[Tensor, "batch seq_len n_heads d_head"]
    ) -> Float[Tensor, "batch seq_len d_model"]:
        self.attn2.key_activations = key_activations
        self.attn2.value_activations = value_activations
        resid_post_causal_attention = self.attn(self.ln1(resid_pre)) + resid_pre
        resid_post_cross_attention = self.attn2(self.ln2(resid_post_causal_attention)) + resid_post_causal_attention
        resid_post_mlp = self.mlp(self.ln2(resid_post_cross_attention)) + resid_post_cross_attention
        return resid_post_mlp

## Unembedding Module

In [ ]:
class Unembed(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_U = nn.Parameter(t.empty(cfg.d_model, cfg.d_vocab))
        self.b_U = nn.Parameter(t.zeros(cfg.d_vocab), requires_grad = False)
        nn.init.normal_(self.W_U, std = self.cfg.init_range)
        
    def forward(
        self, resid_stream: Float[Tensor, "batch seq_len d_model"]
    ) -> Float[Tensor, "batch seq_len d_vocab"]:
        
        Unembedding = einops.einsum(
            resid_stream,
            self.W_U,
            "batch seq_len d_model, d_model d_vocab -> batch seq_len d_vocab") + self.b_U
        return Unembedding

## Full Transformer

In [ ]:
class DemoTransformer(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.embed = Embed(cfg)
        self.pos_embed = PosEmbed(cfg)
        self.encoder_blocks = nn.ModuleList([EncoderBlock(cfg) for _ in range(cfg.n_layers)])
        self.decoder_blocks = nn.ModuleList([DecoderBlock(cfg) for _ in range(cfg.n_layers)])
        self.ln_final = LayerNorm(cfg)
        self.unembed = Unembed(cfg)
    
    def forward(self, tokens: Float[Tensor, "batch seq_len"]
               ) -> Float[Tensor, "batch seq_len d_vocab"]:
        
        residual = self.embed(tokens) + self.pos_embed(tokens)
        for block in self.encoder_blocks:
            residual, key_activations, value_activations = block(residual)
        for block in self.decoder_blocks:
            residual = block(residual, key_activations, value_activations)
        logits = self.unembed(self.ln_final(residual))
        return logits

In [ ]:
demo_transformer = DemoTransformer(Config).to(device)

# Model Configs

In [ ]:
model_cfg = Config(
    d_model=256,
    n_heads=4,
    d_head=64,
    d_mlp=1024,
    n_layers=2,
    n_ctx=256,
    d_vocab= 50257
)
model = DemoTransformer(model_cfg)

@dataclass
class TransformerTrainingArgs():
    batch_size = 16
    epochs = 5
    max_steps_per_epoch = 100
    lr = 1e-3
    weight_decay = 1e-2
    wandb_project: Optional[str] = "day2-demotransformer"
    wandb_name: Optional[str] = 'shaheen-ahmed'

args = TransformerTrainingArgs()

# Loss function

# Training Loop